In [3]:
import sys
from datetime import date
import pandas as pd
from jugaad_data.nse import stock_df

C:\Users\rishi\AppData\Local\Temp\ipykernel_10568\2636643901.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [31]:
def get_ewm(DATA, i, long_term, alpha):
    if (i == 0):
        return DATA[0]
    
    prev_ewm = get_ewm(DATA, i-1, long_term, alpha)
    ewm = alpha*(DATA[i] - prev_ewm) + prev_ewm
    
    return ewm

In [38]:
def macd(path):
    DATA = pd.read_csv(path)
    stocks = 0
    cashflow = 0
    long_ewm = DATA['CLOSE'].ewm(span=26, adjust = False).mean()
    short_ewm = DATA['CLOSE'].ewm(span = 12, adjust = False).mean()
    macd_line = short_ewm - long_ewm
    signal = macd_line.ewm(span = 9, adjust = False).mean()
    for i in range(247):
        man_long_ewm = get_ewm(DATA['CLOSE'], i, 26, (2/27))
        man_short_ewm = get_ewm(DATA['CLOSE'], i, 12, (2/13))
        man_signal = get_ewm(macd_line, i, 9, (2/10))
        if macd_line[i] > signal[i] and stocks < 3:
            stocks += 1
            cashflow -= DATA['CLOSE'][i]
        elif macd_line[i] < signal[i] and stocks > -3:
            stocks -= 1
            cashflow += DATA['CLOSE'][i]
        print(f"{DATA['DATE'][i]} {cashflow}")

In [39]:


# ------------------------------------ generation of data frame --------------------------------------------
def generate_dataframe(symbol, today, lastday, train_data):
    # formatting the date
    to_day = int(today[:2])
    to_month = int(today[3:5])
    to_year = int(today[6:10])
    
    la_day = int(lastday[:2])
    la_month = int(lastday[3:5])
    la_year = int(lastday[6:10])
    
    df = pd.DataFrame(stock_df(symbol=symbol, from_date=date(to_year, to_month, to_day), to_date=date(la_year, la_month, la_day), series="EQ"))
    df = df[[ "DATE", "CLOSE", "HIGH", "LOW", "PREV. CLOSE", "VWAP", "NO OF TRADES", "OPEN"]]
    df = df.iloc[::-1]
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%m-%Y')
    df['DATE'] = df['DATE'].dt.strftime('%d/%m/%Y')
    if (train_data == "0"):
        df.to_csv("Stocks/"+symbol+".csv", index=False)
    else:
        df.to_csv("Stocks/"+symbol+"_train.csv", index = False)

# ----------------------------------------------- pickle --------------------------------------------------------
def write_pickle(DATA, symbol):
    pd.to_pickle(DATA, symbol + ".pkl")

# -------------------------------------------------------- MAIN -----------------------------------------------------------------------
def main():
    argument = "SBIN"
    lastday = "01/01/2024"
    today = "01/01/2023"
    train_data = "0"

    generate_dataframe(argument, today, lastday, train_data)
    #write_pickle(DATA, "Stocks/" + argument)
    macd("Stocks/SBIN.csv")

if __name__ == "__main__":
    main()

02/01/2023 0
03/01/2023 -612.4
04/01/2023 -7.199999999999932
05/01/2023 597.9000000000001
06/01/2023 1198.5500000000002
09/01/2023 1806.2500000000002
10/01/2023 1806.2500000000002
11/01/2023 1806.2500000000002
12/01/2023 1806.2500000000002
13/01/2023 1806.2500000000002
16/01/2023 1806.2500000000002
17/01/2023 1806.2500000000002
18/01/2023 1806.2500000000002
19/01/2023 1806.2500000000002
20/01/2023 1806.2500000000002
23/01/2023 1806.2500000000002
24/01/2023 1806.2500000000002
25/01/2023 1806.2500000000002
27/01/2023 1806.2500000000002
30/01/2023 1806.2500000000002
31/01/2023 1806.2500000000002
01/02/2023 1806.2500000000002
02/02/2023 1806.2500000000002
03/02/2023 1806.2500000000002
06/02/2023 1806.2500000000002
07/02/2023 1806.2500000000002
08/02/2023 1806.2500000000002
09/02/2023 1806.2500000000002
10/02/2023 1253.2000000000003
13/02/2023 715.7000000000003
14/02/2023 174.85000000000025
15/02/2023 -367.39999999999975
16/02/2023 -907.4999999999998
17/02/2023 -1438.4999999999998
20/02/202